In [1]:
import sys
sys.path.append(r'C:\Users\Lina\Documents\Lab-I---Dimensionality-Reduction\Unsupervised_model')

from SVD import SVD as MySVD
from PCA import PCA as MyPCA
from TSNE import TSNE as MyTSNE

import json
import numpy as np
from flask import Flask, request
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


In [2]:
mnist = fetch_openml('mnist_784')
mnist.target = mnist.target.astype(int)
X, y = mnist['data'], mnist['target']

c:\Users\Lina\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
#Reducimos la dimensionalidad de los datos de entrada usando nuestro paquete Python no supervisado:
n_components = 20  # número de componentes para reducción de dimensionalidad
pca = MyPCA(n_components=20)
X_pca_fit = pca.fit(X)
X_pca = pca.fit_transform(X)

Al estandarizar los datos, se asegura que todos los datos de entrada estén en una escala similar y se reduce la probabilidad de que se produzcan valores no finitos durante el entrenamiento del modelo

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_pca)

In [5]:
#Entrenamos el clasificador de dígitos MNIST
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4, solver='sgd', verbose=10, tol=1e-4, random_state=1, learning_rate_init=.1)
clf.fit(X_train_scaled, y)

Iteration 1, loss = 0.34222076
Iteration 2, loss = 0.17682216
Iteration 3, loss = 0.14713743
Iteration 4, loss = 0.13169248
Iteration 5, loss = 0.12367087
Iteration 6, loss = 0.11799772
Iteration 7, loss = 0.11424112
Iteration 8, loss = 0.11225227
Iteration 9, loss = 0.10941752
Iteration 10, loss = 0.10718213


c:\Users\Lina\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(50,), learning_rate_init=0.1, max_iter=10,
              random_state=1, solver='sgd', verbose=10)

In [6]:
app = Flask(__name__)

@app.route('/mnist', methods=['POST'])
def classify_mnist():
    # extraemos la imagen de la solicitud POST
    data = request.json['data']
    
    # convertimos la imagen en una matriz numpy y la redimensionamos
    X = np.array(data).reshape(1, -1)
    
    # reducimos la dimensionalidad de la imagen usando PCA
    X_pca = pca.transform(X)
    
    # predecimos la clase de la imagen usando el clasificador pre-entrenado
    y_pred = clf.predict(X_pca)
    
    # devolvemos la clase predicha como respuesta JSON
    return json.dumps({'class': y_pred.tolist()[0]})

In [7]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
